In [1]:
import pandas as pd
import numpy as np
import datetime

#import xgboost as xgb

from keras.models import Sequential
from keras.layers import Dense
from keras import layers
from keras.layers import Dropout

import matplotlib.pyplot as plt
from tensorflow import keras

In [2]:

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
import torch
import transformers as ppb
import warnings
warnings.filterwarnings('ignore')

In [3]:
from transformers import (
   BertTokenizerFast,
   AutoModelForMaskedLM,
   AutoModelForTokenClassification,
)

tokenizer = BertTokenizerFast.from_pretrained('bert-base-chinese')
model = AutoModelForMaskedLM.from_pretrained('ckiplab/albert-tiny-chinese') # or other models above

In [4]:
#this is the raw dataset
data16 = pd.read_csv(r'/Users/cairo/Google Drive/wechat data/016.csv', sep=',', error_bad_lines=False, index_col=False, dtype='unicode')
#topicdata = pd.read_csv(r'C:/Users/Junhao/Google Drive/wechat data/TopicOutcomeAll20Topic.csv', sep=',', error_bad_lines=False, index_col=False, dtype='unicode')

In [5]:
#this dataset is the raw data processed with cutting word package jieba(a popular word cutting tool for chinese)
#cutword = pd.read_pickle("C:/Users/Junhao/Google Drive/wechat data/cut001.pkl")
cutword = pd.read_pickle("/Users/cairo/Google Drive/wechat data/cut001.pkl")

In [5]:
#this is the dataset with topic modeling output
topicdata = pd.read_csv(r'/Users/cairo/Google Drive/wechat data/TopicOutcomeAll20Topic.csv', sep=',', error_bad_lines=False, index_col=False, dtype='unicode')
#topicdata = pd.read_csv(r'C:/Users/Junhao/Google Drive/wechat data/TopicOutcomeAll20Topic.csv', sep=',', error_bad_lines=False, index_col=False, dtype='unicode')

In [10]:
list(cutword.columns) 

['name',
 'account',
 'author',
 'type',
 'title',
 'summary',
 'likeCount',
 'clicksCount',
 'url',
 'publicTime',
 'orderNum',
 'originalFlag',
 'imageUrl',
 'sourceUrl',
 'videoUrl',
 'musicUrl',
 'audioUrl',
 'updateTime',
 'insertTime',
 'wordsCut']

In [11]:
list(topicdata.columns)

['name',
 'account',
 'author',
 'type',
 'title',
 'summary',
 'likeCount',
 'clicksCount',
 'url',
 'publicTime',
 'orderNum',
 'originalFlag',
 'imageUrl',
 'sourceUrl',
 'videoUrl',
 'musicUrl',
 'audioUrl',
 'updateTime',
 'insertTime',
 'topic0',
 'topic1',
 'topic2',
 'topic3',
 'topic4',
 'topic5',
 'topic6',
 'topic7',
 'topic8',
 'topic9',
 'topic10',
 'topic11',
 'topic12',
 'topic13',
 'topic14',
 'topic15',
 'topic16',
 'topic17',
 'topic18',
 'topic19']

In [37]:
list(data16.columns)

['name',
 'account',
 'author',
 'type',
 'title',
 'summary',
 'content',
 'likeCount',
 'clicksCount',
 'url',
 'publicTime',
 'orderNum',
 'originalFlag',
 'imageUrl',
 'sourceUrl',
 'videoUrl',
 'musicUrl',
 'audioUrl',
 'updateTime',
 'insertTime']

In [25]:
#cutword1 = cutword.head(1000).wordsCut
#cutword3 = cutword.head(1000).title  #i can also try to use only the article titles as bert input 

cutword2 = data16.head(1000).content.dropna() #use 1000 article content from rawdata as bert input


In [26]:
#I need to cut each article to the length of 100 characters because the tokenizer can only deal with a max length of 512
#the computation takes too long if the kept article length is long

cutword3 = []
for i in cutword2:
    dd = i[:100]
    cutword3.append(dd)

In [11]:
cutword3[0:5]

['今天最多人讨论的就是姚贝娜去世的新闻，满屏的悼念。但是她离开的杀手，大家是否足够了解？和梅艳芳同被乳腺癌夺取年轻的生命。今天转发一个视频希望更多女性重视这个疾病。［一分钟远离乳腺癌］VIDEO提起乳腺癌，我们总能想起一大串熟悉的名字，陈晓旭、叶凡、蔡琴、秀兰·邓波儿……她们中有的人战胜了乳腺癌，顽强地活着；而有的则被乳腺癌带走了。越来越多的媒体、政要、名人、明星开始关注乳腺癌。世界头号女性杀手乳腺癌每年夺取50万女性的生命，并且全球每年还有120万新发病例。在我国，北京、上海、天津等大城市的统计显示，乳腺癌是我国女性最常见的恶性肿瘤，且发病率呈逐年上升的趋势。上海是乳腺癌高发病地区，年龄最大的',
 '传递健康资讯 成就美好人生 今天不养生，明天养医生。请将健康观念传递给更多朋友。 更多精彩内容请添加小编个人微信：13861899065山有山的高度，水有水的深度，没必要攀比，每个人都有自己的长处；风有风的自由，云有云的温柔，没必要模仿，每个人都有自己的个性。你认为快乐的，就去寻找；你认为值得的，就去守候；你认为幸福的，就去珍惜。没有不被评说的事，没有不被猜测的人。做最真实最漂亮的自己，依心而行，无憾今生。 人生1条路：走自己的路；人生2件宝：身体好、心不老；人生3种朋友：肯借钱给你、参加你的婚礼、参加你的葬礼；人生有4苦：看不透、舍不得、输不起、放不下。人生5句话：再难也要坚持，再好也要淡泊',
 '↑↑↑巴西插画师 Butcher Billy 将世界各国政商圈的大爷们变成美漫里的大反派——而且看上去还挺符合他们的。他将这个系列命名为现实世界的“复仇者联盟”。Billy除了是个优秀的插画师，他还是个丧心病狂的淘宝店主：他的网店除了让漫迷和极客们疯狂掏钱包的T恤、环保袋、iphone case等等周边，连BB穿的衣服都不放过，当然，上面印的全是些DC的大坏蛋。MAO AS DARKSEID 毛泽东 饰 达克赛德DARKSEID（达克赛德），黑暗君主。使用反生命方程式的他是个多元宇宙级别的存在，可以控制整个多元宇宙的生命。除去可怕的物理能力外，还有着强大的心灵能力，可以控制整个星球的人，还可以',
 '传递健康资讯 成就美好人生 今天不养生，明天养医生。请将健康观念传递给更多朋友。 更多精彩内容请添加小编个人微信：13861899065（图为癌症患者绝望的流泪）所有

In [27]:
cutword3 = pd.Series(cutword3)
tokenized = cutword3.apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))

In [116]:
cutword3.shape

(998,)

In [28]:
#padding 
max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)

padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])
padded.shape

(998, 102)

In [29]:
#masking
attention_mask = np.where(padded != 0, 1, 0)
attention_mask.shape

(998, 102)

In [30]:
input_ids = torch.tensor(padded)  
input_ids = torch.tensor(input_ids).to(torch.int64)
attention_mask = torch.tensor(attention_mask)


In [31]:
attention_mask.shape

torch.Size([998, 102])

In [32]:
input_ids.shape

torch.Size([998, 102])

In [33]:
with torch.no_grad():
    last_hidden_states = model(input_ids, attention_mask=attention_mask)

In [34]:
features = last_hidden_states[0][:,0,:].numpy()

In [35]:
likeCount = data16.head(1000).likeCount #use likecount as dependent variable 

df = data16[["likeCount", "content"]].head(1000)

removeinds = pd.isnull(df).any(1).to_numpy().nonzero()[0]

likeCount = [i for j, i in enumerate(likeCount) if j not in removeinds] # remove the two likecount where content is empty


In [36]:
train_features, test_features, train_output, test_output = train_test_split(features, likeCount)

In [55]:
#training error
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error


reg = LinearRegression().fit(train_features, train_output)
#reg.score(train_features, train_labels)
preds = reg.predict(train_features)


rmse = np.sqrt(mean_squared_error(np.array(train_output).astype(float), preds))
print("RMSE: %f" % (rmse))

print("R square: %f" % (r2_score(np.array(train_output).astype(float), preds)))

RMSE: 202.018867
R square: -0.119434


In [57]:
from statsmodels.formula.api  import ols
import statsmodels.api as sm
results = sm.OLS(np.array(train_output).astype(float), train_features.astype(float)).fit()
results.summary().tables[0]

Dep. Variable:,y,R-squared:,0.986
Model:,OLS,Adj. R-squared:,0.667
Method:,Least Squares,F-statistic:,3.089
Date:,"Sun, 24 Jan 2021",Prob (F-statistic):,0.000120
Time:,20:27:54,Log-Likelihood:,-3400.8
No. Observations:,748,AIC:,8234.
Df Residuals:,32,BIC:,1.154e+04
Df Model:,715,,
Covariance Type:,nonrobust,,


In [59]:
#test error
preds = reg.predict(test_features)

rmse = np.sqrt(mean_squared_error(np.array(test_output).astype(float), preds))
print("RMSE: %f" % (rmse))

print("R square: %f" % (r2_score(np.array(test_output).astype(float), preds)))

RMSE: 242.183767
R square: -0.650406
